In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# expressed_gene_tpm_threshold = -1
# minimum_samples_threshold_long = -1
# minimum_samples_threshold_circadian = -1

In [ ]:
transcriptome = pd.read_csv(
    "/Users/michael/Data/Luke_terrace_experiment/General_data/plate1_4_tpm_normalized.csv",
    index_col=0,
)
rows_to_drop_expression_data = [
    "A2450525897_n01_undetermined",
    "A2449446903_n01_undetermined",
    "B250508004_n01_undetermined",
    "B2449500127_n01_undetermined",
]
transcriptome = transcriptome.drop(index=rows_to_drop_expression_data)
transcriptome = transcriptome.sort_index()

metadata = pd.read_csv(
    "/Users/michael/Data/Luke_terrace_experiment/General_data/Plates_1_to_4_metadata_merged_luke.csv",
    index_col=0,
)
metadata = metadata.drop(
    columns=[
        "arb.sort",
        "sample-id",
        "Ambiguous Unstranded",
        "Ambiguous Forward",
        "Multimapping",
        "Unmapped Over Mapped",
    ]
)
metadata["Date and Time"] = metadata["date"] + " " + metadata["time"]
luke_time_data_format = "%-m/%-d/%y %-H:%-M"
metadata["Date and Time"] = pd.to_datetime(
    metadata["Date and Time"], format=luke_time_data_format
)

unusable_samples = [
    "A2450525897_n01_LICRNA01_A06",
    "A2450525897_n01_LICRNA01_A08",
    "A2450525897_n01_LICRNA01_C11",
    "A2450525897_n01_LICRNA01_D11",
    "A2450525897_n01_LICRNA01_D12",
    "A2449446903_n01_LICRNA02_F01",
    "B2449500127_n01_LICRNA04_A04",
    "B2449500127_n01_LICRNA04_A06",
    "B2449500127_n01_LICRNA04_A07",
]
borderline_unusable = [
    "A2450525897_n01_LICRNA01_F03",
    "A2450525897_n01_LICRNA01_G07",
    "A2449446903_n01_LICRNA02_A04",
    "A2449446903_n01_LICRNA02_H10",
    "B2449500127_n01_LICRNA04_H02",
    "B2449500127_n01_LICRNA04_H11",
]
all_unsable = borderline_unusable + unusable_samples

trimmed_transcriptome = transcriptome.drop(index=all_unsable)
trimmed_metadata = metadata.drop(index=all_unsable)
trimmed_metadata = trimmed_metadata.sort_index()
trimmed_transcriptome = trimmed_transcriptome.dropna(how="all", axis=0)
trimmed_transcriptome = trimmed_transcriptome.dropna(how="all", axis=1)

In [ ]:
# trimmed_transcriptome = trimmed_transcriptome.loc[
#     :, trimmed_transcriptome.sum() > expressed_gene_tpm_threshold
# ]

# trimmed_transcriptome

In [ ]:
long_term_transcriptome = trimmed_transcriptome.loc[
    trimmed_metadata["Experiment Type"] == "Long Term"
]
long_term_metadata = trimmed_metadata.loc[
    trimmed_metadata["Experiment Type"] == "Long Term"
]

In [ ]:
trimmed_metadata

In [ ]:
# circadian_metadata_1 = trimmed_metadata.loc[
#     trimmed_metadata["Experiment Type"] == ("Circadian Experiment 1")
# ]
# circadian_metadata_2 = trimmed_metadata.loc[
#     trimmed_metadata["Experiment Type"] == ("Circadian Experiment 2")
# ]

# circadian_transcriptome_1 = trimmed_transcriptome.loc[
#     trimmed_metadata["Experiment Type"] == ("Circadian Experiment 1")
# ]
# circadian_transcriptome_2 = trimmed_transcriptome.loc[
#     trimmed_metadata["Experiment Type"] == ("Circadian Experiment 2")
# ]

In [ ]:
long_term_transcriptome

In [ ]:
long_term_metadata

In [ ]:
long_term_transcriptome = long_term_transcriptome.sort_index()
long_term_metadata = long_term_metadata.sort_index()

long_term_transcriptome

In [ ]:
microbiome_abundance = pd.read_csv(
    "/Users/michael/Data/Luke_terrace_experiment/Microbiome/lic2024_16S_rab.csv"
)
merged_microbiome = microbiome_abundance.merge(
    metadata[["sampID", "Experiment Type"]], left_on="plantID", right_on="sampID"
)
long_term_microbiome = merged_microbiome.loc[
    merged_microbiome["Experiment Type"] == "Long Term"
]
long_term_microbiome

In [ ]:
# circadian_microbiome_1 = merged_microbiome.loc[
#     merged_microbiome["Experiment Type"] == "Circadian Experiment 1"
# ]

# circadian_microbiome_1 = merged_microbiome.loc[
#     merged_microbiome["Experiment Type"] == "Circadian Experiment 1"
# ]

In [ ]:
merged_microbiome

In [ ]:
genus_sum = (
    long_term_microbiome[["Sample", "Genus", "AbundR100"]]
    .groupby(["Sample", "Genus"])
    .sum()
)
genus_sum = genus_sum.reset_index()

genus_sum_w_tp = genus_sum.merge(
    metadata[["sampID", "timepoint"]], left_on="Sample", right_on="sampID"
)
genus_sum_w_tp

In [ ]:
genus_sum.set_index(
    "Sample",
    drop=True,
).index.difference(
    genus_sum_w_tp.set_index("Sample", drop=True).index
)  ## Difference is from the dropped samples (due to low quality RNA) which we dont' wanna drop here cause its DNA

In [ ]:
genus_sum_w_tp.head(40)

In [ ]:
all_present_genus = genus_sum_w_tp["Genus"].unique()
all_present_genus

In [ ]:
## Need to fill in missing genus in every sample first
for sampID in genus_sum_w_tp["sampID"].unique():
    sampid_only_df = genus_sum_w_tp.loc[genus_sum_w_tp["sampID"] == sampID]
    present_genus_in_samp = sampid_only_df["Genus"].unique()
    difference_in_present_bacteria = list(
        set(all_present_genus) - set(present_genus_in_samp)
    )
    current_time_point = sampid_only_df["timepoint"].iloc[0]
    empty_df_to_add = pd.DataFrame(
        columns=["Sample", "Genus", "AbundR100", "sampID", "timepoint"]
    )
    empty_df_to_add["Genus"] = difference_in_present_bacteria
    empty_df_to_add["Sample"] = sampID
    empty_df_to_add["AbundR100"] = 0
    empty_df_to_add["sampID"] = sampID
    empty_df_to_add["timepoint"] = current_time_point
    genus_sum_w_tp = pd.concat([genus_sum_w_tp, empty_df_to_add])

In [ ]:
timepoint_average_abundance = (
    genus_sum_w_tp[["Genus", "timepoint", "AbundR100"]]
    .groupby(["Genus", "timepoint"])
    .mean()
)
timepoint_average_abundance = timepoint_average_abundance.reset_index()
timepoint_average_abundance

In [ ]:
# Check structure of the key dataframes
print("long_term_metadata columns:", long_term_metadata.columns.tolist())
print(
    "long_term_metadata timepoint unique values:",
    sorted(long_term_metadata["timepoint"].unique()),
)
print(
    "\ntimepoint_average_abundance columns:",
    timepoint_average_abundance.columns.tolist(),
)
print(
    "timepoint_average_abundance timepoint unique values:",
    sorted(timepoint_average_abundance["timepoint"].unique()),
)

In [ ]:
# First, let's aggregate transcriptome data by timepoint (averaging across samples)
long_term_transcriptome_w_tp = long_term_transcriptome.copy()
long_term_transcriptome_w_tp["timepoint"] = long_term_metadata["timepoint"]

# Average gene expression by timepoint
transcriptome_by_timepoint = long_term_transcriptome_w_tp.groupby("timepoint").mean()
transcriptome_by_timepoint

In [ ]:
# Get sorted unique timepoints
timepoints = sorted(transcriptome_by_timepoint.index.unique())
print(f"Available timepoints: {timepoints}")
print(f"Number of timepoints: {len(timepoints)}")

In [ ]:
# Create shifted transcriptome dataframes
# Shift forward: transcriptome at t correlates with microbiome at t+1 (transcriptome leads)
# Shift backward: transcriptome at t correlates with microbiome at t-1 (microbiome leads)

# Create a mapping from original timepoint to shifted timepoint
tp_to_idx = {tp: i for i, tp in enumerate(timepoints)}
idx_to_tp = {i: tp for i, tp in enumerate(timepoints)}

# Shift forward (+1): transcriptome data moved to next timepoint
# This tests: does expression at time t correlate with microbiome at time t+1?
transcriptome_shifted_forward = transcriptome_by_timepoint.copy()
transcriptome_shifted_forward.index = [
    idx_to_tp.get(tp_to_idx[tp] + 1, None) for tp in transcriptome_shifted_forward.index
]
transcriptome_shifted_forward = transcriptome_shifted_forward.dropna(subset=[])
transcriptome_shifted_forward = transcriptome_shifted_forward[
    transcriptome_shifted_forward.index.notna()
]

# Shift backward (-1): transcriptome data moved to previous timepoint
# This tests: does expression at time t correlate with microbiome at time t-1?
transcriptome_shifted_backward = transcriptome_by_timepoint.copy()
transcriptome_shifted_backward.index = [
    idx_to_tp.get(tp_to_idx[tp] - 1, None)
    for tp in transcriptome_shifted_backward.index
]
transcriptome_shifted_backward = transcriptome_shifted_backward[
    transcriptome_shifted_backward.index.notna()
]

print(f"Original transcriptome timepoints: {list(transcriptome_by_timepoint.index)}")
print(f"Forward shifted timepoints: {list(transcriptome_shifted_forward.index)}")
print(f"Backward shifted timepoints: {list(transcriptome_shifted_backward.index)}")

In [ ]:
# Pivot microbiome data to have timepoints as rows and genus as columns
microbiome_by_timepoint = timepoint_average_abundance.pivot(
    index="timepoint", columns="Genus", values="AbundR100"
)
microbiome_by_timepoint = microbiome_by_timepoint.sort_index()
microbiome_by_timepoint

In [ ]:
# microbiome_by_timepoint = microbiome_by_timepoint.loc[
#     :, (microbiome_by_timepoint > 0).sum() > minimum_samples_threshold_long
# ]

In [ ]:
def calculate_gene_genus_correlations(transcriptome_df, microbiome_df):
    """
    Calculate correlations between all genes and all genera across shared timepoints.
    Returns a DataFrame with genes as rows and genera as columns.
    Uses vectorized Spearman correlation via pandas for speed.
    """
    from scipy.stats import spearmanr

    # Find shared timepoints
    shared_timepoints = transcriptome_df.index.intersection(microbiome_df.index)

    # Subset to shared timepoints
    trans_subset = transcriptome_df.loc[shared_timepoints]
    micro_subset = microbiome_df.loc[shared_timepoints]

    print(f"Calculating correlations across {len(shared_timepoints)} shared timepoints")

    # Vectorized approach: compute all correlations at once using scipy's spearmanr
    # spearmanr can take two 2D arrays and return correlation matrix between all columns
    trans_values = trans_subset.values  # shape: (n_timepoints, n_genes)
    micro_values = micro_subset.values  # shape: (n_timepoints, n_genera)

    # Concatenate and compute full correlation matrix, then extract the cross-correlation block
    combined = np.hstack([trans_values, micro_values])
    n_genes = trans_values.shape[1]
    n_genera = micro_values.shape[1]

    # spearmanr returns correlation matrix and p-value matrix
    corr_full, pval_full = spearmanr(combined, axis=0)

    # Extract the cross-correlation block (genes x genera)
    correlation_matrix = pd.DataFrame(
        corr_full[:n_genes, n_genes:],
        index=trans_subset.columns,
        columns=micro_subset.columns,
    )
    pvalue_matrix = pd.DataFrame(
        pval_full[:n_genes, n_genes:],
        index=trans_subset.columns,
        columns=micro_subset.columns,
    )

    return correlation_matrix, pvalue_matrix, shared_timepoints

In [ ]:
# Calculate correlations for all three scenarios
print("=== No shift (synchronous) ===")
corr_no_shift, pval_no_shift, tp_no_shift = calculate_gene_genus_correlations(
    transcriptome_by_timepoint, microbiome_by_timepoint
)

print("\n=== Forward shift (transcriptome leads microbiome by 1 timepoint) ===")
corr_forward, pval_forward, tp_forward = calculate_gene_genus_correlations(
    transcriptome_shifted_forward, microbiome_by_timepoint
)

print("\n=== Backward shift (microbiome leads transcriptome by 1 timepoint) ===")
corr_backward, pval_backward, tp_backward = calculate_gene_genus_correlations(
    transcriptome_shifted_backward, microbiome_by_timepoint
)

In [ ]:
import pickle
import os

# # Create output directory if it doesn't exist
output_dir = "/Users/michael/Data/Luke_terrace_experiment/General_data/Junk_data/Time_shifted_correlation"
os.makedirs(output_dir, exist_ok=True)

# # Save no shift correlations
# dbfile = open(f"{output_dir}/corr_no_shift_remove_low_exp.p", "ab")
# pickle.dump(corr_no_shift, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/pval_no_shift_remove_low_exp.p", "ab")
# pickle.dump(pval_no_shift, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/tp_no_shift_remove_low_exp.p", "ab")
# pickle.dump(tp_no_shift, dbfile)
# dbfile.close()

# # Save forward shift correlations
# dbfile = open(f"{output_dir}/corr_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(corr_forward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/pval_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(pval_forward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/tp_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(tp_forward, dbfile)
# dbfile.close()

# # Save backward shift correlations
# dbfile = open(f"{output_dir}/corr_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(corr_backward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/pval_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(pval_backward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/tp_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(tp_backward, dbfile)
# dbfile.close()

# print(f"Saved all correlation results to {output_dir}")

# Read no shift correlations
# dbfile = open(f"{output_dir}/corr_no_shift.p", "rb")
# corr_no_shift = pickle.load(dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/pval_no_shift.p", "rb")
# pval_no_shift = pickle.load(dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/tp_no_shift.p", "rb")
# tp_no_shift = pickle.load(dbfile)
# dbfile.close()

# # Read forward shift correlations
# dbfile = open(f"{output_dir}/corr_forward_shift.p", "rb")
# corr_forward = pickle.load(dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/pval_forward_shift.p", "rb")
# pval_forward = pickle.load(dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/tp_forward_shift.p", "rb")
# tp_forward = pickle.load(dbfile)
# dbfile.close()

# # Read backward shift correlations
# dbfile = open(f"{output_dir}/corr_backward_shift.p", "rb")
# corr_backward = pickle.load(dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/pval_backward_shift.p", "rb")
# pval_backward = pickle.load(dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir}/tp_backward_shift.p", "rb")
# tp_backward = pickle.load(dbfile)
# dbfile.close()

# print(f"Loaded all correlation results from {output_dir}")

In [ ]:
# Compare correlation distributions across shifts
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Flatten correlation matrices for distribution plots
corr_no_shift_flat = corr_no_shift.values.flatten()
corr_forward_flat = corr_forward.values.flatten()
corr_backward_flat = corr_backward.values.flatten()

# Remove NaN values
corr_no_shift_flat = corr_no_shift_flat[~np.isnan(corr_no_shift_flat)]
corr_forward_flat = corr_forward_flat[~np.isnan(corr_forward_flat)]
corr_backward_flat = corr_backward_flat[~np.isnan(corr_backward_flat)]

axes[0].hist(corr_no_shift_flat, bins=50, alpha=0.7, edgecolor="black")
axes[0].set_title("No Shift (Synchronous)")
axes[0].set_xlabel("Spearman Correlation")
axes[0].set_ylabel("Frequency")
axes[0].axvline(x=0, color="red", linestyle="--", alpha=0.5)

axes[1].hist(corr_forward_flat, bins=50, alpha=0.7, edgecolor="black", color="green")
axes[1].set_title("Forward Shift\n(Transcriptome leads)")
axes[1].set_xlabel("Spearman Correlation")
axes[1].set_ylabel("Frequency")
axes[1].axvline(x=0, color="red", linestyle="--", alpha=0.5)

axes[2].hist(corr_backward_flat, bins=50, alpha=0.7, edgecolor="black", color="orange")
axes[2].set_title("Backward Shift\n(Microbiome leads)")
axes[2].set_xlabel("Spearman Correlation")
axes[2].set_ylabel("Frequency")
axes[2].axvline(x=0, color="red", linestyle="--", alpha=0.5)

plt.tight_layout()
plt.show()

print(f"No shift - Mean abs correlation: {np.mean(np.abs(corr_no_shift_flat)):.4f}")
print(f"Forward shift - Mean abs correlation: {np.mean(np.abs(corr_forward_flat)):.4f}")
print(
    f"Backward shift - Mean abs correlation: {np.mean(np.abs(corr_backward_flat)):.4f}"
)

In [ ]:
# Find genes with strongest delayed correlations
# Compare: which genes have better correlations with shifted vs non-shifted data?


def find_best_shift_correlations(
    corr_no_shift, corr_forward, corr_backward, pval_threshold=0.05
):
    """
    For each gene-genus pair, find which shift gives the strongest correlation.
    """
    results = []

    for gene in corr_no_shift.index:
        for genus in corr_no_shift.columns:
            r_sync = corr_no_shift.loc[gene, genus]
            r_fwd = (
                corr_forward.loc[gene, genus] if gene in corr_forward.index else np.nan
            )
            r_bwd = (
                corr_backward.loc[gene, genus]
                if gene in corr_backward.index
                else np.nan
            )

            # Find the shift with max absolute correlation
            correlations = {"sync": r_sync, "forward": r_fwd, "backward": r_bwd}
            valid_corrs = {k: v for k, v in correlations.items() if not np.isnan(v)}

            if valid_corrs:
                best_shift = max(valid_corrs.keys(), key=lambda k: abs(valid_corrs[k]))
                best_corr = valid_corrs[best_shift]

                results.append(
                    {
                        "gene": gene,
                        "genus": genus,
                        "best_shift": best_shift,
                        "best_correlation": best_corr,
                        "sync_correlation": r_sync,
                        "forward_correlation": r_fwd,
                        "backward_correlation": r_bwd,
                    }
                )

    return pd.DataFrame(results)


shift_comparison = find_best_shift_correlations(
    corr_no_shift, corr_forward, corr_backward
)
shift_comparison

In [ ]:
# Summary: count how many gene-genus pairs have each shift as their best
shift_counts = shift_comparison["best_shift"].value_counts()
print("Best shift distribution across all gene-genus pairs:")
print(shift_counts)
print(f"\nTotal pairs analyzed: {len(shift_comparison)}")

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
shift_counts.plot(kind="bar", ax=ax, color=["blue", "green", "orange"])
ax.set_title("Distribution of Best Shift Across Gene-Genus Pairs")
ax.set_xlabel("Shift Type")
ax.set_ylabel("Number of Gene-Genus Pairs")
ax.set_xticklabels(
    ["Synchronous", "Transcriptome Leads", "Microbiome Leads"], rotation=45
)
plt.tight_layout()
plt.show()

In [ ]:
# Show top genes with strongest delayed correlations (forward or backward shift better than sync)
# Filter for pairs where shifted correlation is notably stronger

shift_comparison["improvement_forward"] = (
    shift_comparison["forward_correlation"].abs()
    - shift_comparison["sync_correlation"].abs()
)
shift_comparison["improvement_backward"] = (
    shift_comparison["backward_correlation"].abs()
    - shift_comparison["sync_correlation"].abs()
)

# Top genes where transcriptome leads (forward shift is better)
top_forward = shift_comparison.nlargest(20, "improvement_forward")[
    ["gene", "genus", "sync_correlation", "forward_correlation", "improvement_forward"]
]
print(
    "Top 20 gene-genus pairs where transcriptome LEADS microbiome (forward shift better):"
)
display(top_forward)

# Top genes where microbiome leads (backward shift is better)
top_backward = shift_comparison.nlargest(20, "improvement_backward")[
    [
        "gene",
        "genus",
        "sync_correlation",
        "backward_correlation",
        "improvement_backward",
    ]
]
print(
    "\nTop 20 gene-genus pairs where microbiome LEADS transcriptome (backward shift better):"
)
display(top_backward)

In [ ]:
shift_comparison.loc[shift_comparison["genus"] == "Sphingomonas_L_486704"].sort_values(
    by="improvement_forward", ascending=False
).head(30)

In [ ]:
sphingo_only = shift_comparison.loc[
    shift_comparison["genus"] == "Sphingomonas_L_486704"
]

In [ ]:
sphingo_only[
    [
        "sync_correlation",
        "forward_correlation",
        "backward_correlation",
        "improvement_forward",
        "improvement_backward",
    ]
].mean()

In [ ]:
# Heatmap comparison for a selected genus showing correlations across shifts
# Select a few genera with interesting patterns
top_genera = microbiome_by_timepoint.mean().nlargest(5).index.tolist()

fig, axes = plt.subplots(1, 3, figsize=(18, 8))

# For visualization, select top variable genes
gene_variance = transcriptome_by_timepoint.var()
top_variable_genes = gene_variance.nlargest(50).index

for ax, (title, corr_mat) in zip(
    axes,
    [
        ("No Shift (Synchronous)", corr_no_shift),
        ("Forward Shift (Transcriptome leads)", corr_forward),
        ("Backward Shift (Microbiome leads)", corr_backward),
    ],
):
    subset = corr_mat.loc[top_variable_genes, top_genera]
    sns.heatmap(
        subset.astype(float),
        ax=ax,
        cmap="RdBu_r",
        center=0,
        vmin=-1,
        vmax=1,
        cbar_kws={"shrink": 0.8},
    )
    ax.set_title(title)
    ax.set_xlabel("Genus")
    ax.set_ylabel("Gene")

plt.tight_layout()
plt.show()

In [ ]:
# Store the correlation results for further analysis
correlation_results = {
    "no_shift": {"correlations": corr_no_shift, "pvalues": pval_no_shift},
    "forward_shift": {"correlations": corr_forward, "pvalues": pval_forward},
    "backward_shift": {"correlations": corr_backward, "pvalues": pval_backward},
}

# Summary statistics per genus for each shift
print("Average absolute correlation per genus across all shifts:\n")
summary_by_genus = pd.DataFrame(
    {
        "Synchronous": corr_no_shift.abs().mean(),
        "Forward (Trans leads)": corr_forward.abs().mean(),
        "Backward (Micro leads)": corr_backward.abs().mean(),
    }
)
summary_by_genus["Best Shift"] = summary_by_genus.idxmax(axis=1)
display(summary_by_genus.sort_values("Synchronous", ascending=False))

In [ ]:
summary_by_genus.sort_values("Backward (Micro leads)").head(50)

In [ ]:
# Circadian Experiment 1 - Time Shifted Correlation Analysis

In [ ]:
# Prepare Circadian Experiment 1 microbiome data
circadian_microbiome_1 = merged_microbiome.loc[
    merged_microbiome["Experiment Type"] == "Circadian Experiment 1"
]

# Aggregate genus abundance by sample
circ1_genus_sum = (
    circadian_microbiome_1[["Sample", "Genus", "AbundR100"]]
    .groupby(["Sample", "Genus"])
    .sum()
    .reset_index()
)

# Merge with timepoint info
circ1_genus_sum_w_tp = circ1_genus_sum.merge(
    metadata[["sampID", "timepoint"]], left_on="Sample", right_on="sampID"
)

# Fill in missing genus for each sample
all_circ1_genus = circ1_genus_sum_w_tp["Genus"].unique()
for sampID in circ1_genus_sum_w_tp["sampID"].unique():
    sampid_only_df = circ1_genus_sum_w_tp.loc[circ1_genus_sum_w_tp["sampID"] == sampID]
    present_genus_in_samp = sampid_only_df["Genus"].unique()
    difference_in_present_bacteria = list(
        set(all_circ1_genus) - set(present_genus_in_samp)
    )
    current_time_point = sampid_only_df["timepoint"].iloc[0]
    empty_df_to_add = pd.DataFrame(
        columns=["Sample", "Genus", "AbundR100", "sampID", "timepoint"]
    )
    empty_df_to_add["Genus"] = difference_in_present_bacteria
    empty_df_to_add["Sample"] = sampID
    empty_df_to_add["AbundR100"] = 0
    empty_df_to_add["sampID"] = sampID
    empty_df_to_add["timepoint"] = current_time_point
    circ1_genus_sum_w_tp = pd.concat([circ1_genus_sum_w_tp, empty_df_to_add])

# Calculate average abundance by timepoint
circ1_timepoint_avg_abundance = (
    circ1_genus_sum_w_tp[["Genus", "timepoint", "AbundR100"]]
    .groupby(["Genus", "timepoint"])
    .mean()
    .reset_index()
)

print(
    f"Circadian 1 - Unique timepoints: {sorted(circ1_timepoint_avg_abundance['timepoint'].unique())}"
)
print(f"Circadian 1 - Number of genera: {len(all_circ1_genus)}")

In [ ]:
# Aggregate Circadian 1 transcriptome by timepoint
circadian_transcriptome_1 = circadian_transcriptome_1.sort_index()
circadian_metadata_1 = circadian_metadata_1.sort_index()

circ1_transcriptome_w_tp = circadian_transcriptome_1.copy()
circ1_transcriptome_w_tp["timepoint"] = circadian_metadata_1["timepoint"]
circ1_transcriptome_by_tp = circ1_transcriptome_w_tp.groupby("timepoint").mean()

# Get sorted timepoints
circ1_timepoints = sorted(circ1_transcriptome_by_tp.index.unique())
print(f"Circadian 1 transcriptome timepoints: {circ1_timepoints}")

# Create shifted transcriptome dataframes
circ1_tp_to_idx = {tp: i for i, tp in enumerate(circ1_timepoints)}
circ1_idx_to_tp = {i: tp for i, tp in enumerate(circ1_timepoints)}

# Forward shift: transcriptome at t correlates with microbiome at t+1 (transcriptome leads)
circ1_trans_shifted_forward = circ1_transcriptome_by_tp.copy()
circ1_trans_shifted_forward.index = [
    circ1_idx_to_tp.get(circ1_tp_to_idx[tp] + 1, None)
    for tp in circ1_trans_shifted_forward.index
]
circ1_trans_shifted_forward = circ1_trans_shifted_forward[
    circ1_trans_shifted_forward.index.notna()
]

# Backward shift: transcriptome at t correlates with microbiome at t-1 (microbiome leads)
circ1_trans_shifted_backward = circ1_transcriptome_by_tp.copy()
circ1_trans_shifted_backward.index = [
    circ1_idx_to_tp.get(circ1_tp_to_idx[tp] - 1, None)
    for tp in circ1_trans_shifted_backward.index
]
circ1_trans_shifted_backward = circ1_trans_shifted_backward[
    circ1_trans_shifted_backward.index.notna()
]

print(f"Forward shifted timepoints: {list(circ1_trans_shifted_forward.index)}")
print(f"Backward shifted timepoints: {list(circ1_trans_shifted_backward.index)}")

In [ ]:
# Pivot microbiome data for Circadian 1
circ1_microbiome_by_tp = circ1_timepoint_avg_abundance.pivot(
    index="timepoint", columns="Genus", values="AbundR100"
)

circ1_microbiome_by_tp = circ1_microbiome_by_tp.loc[
    :, (circ1_microbiome_by_tp > 0).sum() > minimum_samples_threshold_circadian
]


circ1_microbiome_by_tp = circ1_microbiome_by_tp.sort_index()

# Calculate correlations for Circadian 1
print("=== Circadian 1: No shift (synchronous) ===")
circ1_corr_no_shift, circ1_pval_no_shift, circ1_tp_no_shift = (
    calculate_gene_genus_correlations(circ1_transcriptome_by_tp, circ1_microbiome_by_tp)
)

print("\n=== Circadian 1: Forward shift (transcriptome leads) ===")
circ1_corr_forward, circ1_pval_forward, circ1_tp_forward = (
    calculate_gene_genus_correlations(
        circ1_trans_shifted_forward, circ1_microbiome_by_tp
    )
)

print("\n=== Circadian 1: Backward shift (microbiome leads) ===")
circ1_corr_backward, circ1_pval_backward, circ1_tp_backward = (
    calculate_gene_genus_correlations(
        circ1_trans_shifted_backward, circ1_microbiome_by_tp
    )
)

In [ ]:
# Circadian 1: Compare correlation distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

circ1_corr_no_shift_flat = circ1_corr_no_shift.values.flatten()
circ1_corr_forward_flat = circ1_corr_forward.values.flatten()
circ1_corr_backward_flat = circ1_corr_backward.values.flatten()

circ1_corr_no_shift_flat = circ1_corr_no_shift_flat[~np.isnan(circ1_corr_no_shift_flat)]
circ1_corr_forward_flat = circ1_corr_forward_flat[~np.isnan(circ1_corr_forward_flat)]
circ1_corr_backward_flat = circ1_corr_backward_flat[~np.isnan(circ1_corr_backward_flat)]

axes[0].hist(circ1_corr_no_shift_flat, bins=50, alpha=0.7, edgecolor="black")
axes[0].set_title("Circadian 1: No Shift")
axes[0].set_xlabel("Spearman Correlation")
axes[0].axvline(x=0, color="red", linestyle="--", alpha=0.5)

axes[1].hist(
    circ1_corr_forward_flat, bins=50, alpha=0.7, edgecolor="black", color="green"
)
axes[1].set_title("Circadian 1: Forward Shift\n(Transcriptome leads)")
axes[1].set_xlabel("Spearman Correlation")
axes[1].axvline(x=0, color="red", linestyle="--", alpha=0.5)

axes[2].hist(
    circ1_corr_backward_flat, bins=50, alpha=0.7, edgecolor="black", color="orange"
)
axes[2].set_title("Circadian 1: Backward Shift\n(Microbiome leads)")
axes[2].set_xlabel("Spearman Correlation")
axes[2].axvline(x=0, color="red", linestyle="--", alpha=0.5)

plt.suptitle("Circadian Experiment 1 - Correlation Distributions", y=1.02)
plt.tight_layout()
plt.show()

print(
    f"Circadian 1 - No shift mean abs correlation: {np.mean(np.abs(circ1_corr_no_shift_flat)):.4f}"
)
print(
    f"Circadian 1 - Forward shift mean abs correlation: {np.mean(np.abs(circ1_corr_forward_flat)):.4f}"
)
print(
    f"Circadian 1 - Backward shift mean abs correlation: {np.mean(np.abs(circ1_corr_backward_flat)):.4f}"
)

In [ ]:
# Save Circadian 1 results
output_dir_circ1 = "/Users/michael/Data/Luke_terrace_experiment/General_data/Junk_data/Time_shifted_correlation/Circadian_1"
os.makedirs(output_dir_circ1, exist_ok=True)

# dbfile = open(f"{output_dir_circ1}/corr_no_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_corr_no_shift, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ1}/pval_no_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_pval_no_shift, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ1}/tp_no_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_tp_no_shift, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ1}/corr_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_corr_forward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ1}/pval_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_pval_forward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ1}/tp_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_tp_forward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ1}/corr_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_corr_backward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ1}/pval_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_pval_backward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ1}/tp_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ1_tp_backward, dbfile)
# dbfile.close()

print(f"Saved Circadian 1 results to {output_dir_circ1}")

# Circadian Experiment 2 - Time Shifted Correlation Analysis

In [ ]:
# Prepare Circadian Experiment 2 microbiome data
circadian_microbiome_2 = merged_microbiome.loc[
    merged_microbiome["Experiment Type"] == "Circadian Experiment 2"
]

# Aggregate genus abundance by sample
circ2_genus_sum = (
    circadian_microbiome_2[["Sample", "Genus", "AbundR100"]]
    .groupby(["Sample", "Genus"])
    .sum()
    .reset_index()
)

# Merge with timepoint info
circ2_genus_sum_w_tp = circ2_genus_sum.merge(
    metadata[["sampID", "timepoint"]], left_on="Sample", right_on="sampID"
)

# Fill in missing genus for each sample
all_circ2_genus = circ2_genus_sum_w_tp["Genus"].unique()
for sampID in circ2_genus_sum_w_tp["sampID"].unique():
    sampid_only_df = circ2_genus_sum_w_tp.loc[circ2_genus_sum_w_tp["sampID"] == sampID]
    present_genus_in_samp = sampid_only_df["Genus"].unique()
    difference_in_present_bacteria = list(
        set(all_circ2_genus) - set(present_genus_in_samp)
    )
    current_time_point = sampid_only_df["timepoint"].iloc[0]
    empty_df_to_add = pd.DataFrame(
        columns=["Sample", "Genus", "AbundR100", "sampID", "timepoint"]
    )
    empty_df_to_add["Genus"] = difference_in_present_bacteria
    empty_df_to_add["Sample"] = sampID
    empty_df_to_add["AbundR100"] = 0
    empty_df_to_add["sampID"] = sampID
    empty_df_to_add["timepoint"] = current_time_point
    circ2_genus_sum_w_tp = pd.concat([circ2_genus_sum_w_tp, empty_df_to_add])

# Calculate average abundance by timepoint
circ2_timepoint_avg_abundance = (
    circ2_genus_sum_w_tp[["Genus", "timepoint", "AbundR100"]]
    .groupby(["Genus", "timepoint"])
    .mean()
    .reset_index()
)

print(
    f"Circadian 2 - Unique timepoints: {sorted(circ2_timepoint_avg_abundance['timepoint'].unique())}"
)
print(f"Circadian 2 - Number of genera: {len(all_circ2_genus)}")

In [ ]:
# Aggregate Circadian 2 transcriptome by timepoint
circadian_transcriptome_2 = circadian_transcriptome_2.sort_index()
circadian_metadata_2 = circadian_metadata_2.sort_index()

circ2_transcriptome_w_tp = circadian_transcriptome_2.copy()
circ2_transcriptome_w_tp["timepoint"] = circadian_metadata_2["timepoint"]
circ2_transcriptome_by_tp = circ2_transcriptome_w_tp.groupby("timepoint").mean()

# Get sorted timepoints
circ2_timepoints = sorted(circ2_transcriptome_by_tp.index.unique())
print(f"Circadian 2 transcriptome timepoints: {circ2_timepoints}")

# Create shifted transcriptome dataframes
circ2_tp_to_idx = {tp: i for i, tp in enumerate(circ2_timepoints)}
circ2_idx_to_tp = {i: tp for i, tp in enumerate(circ2_timepoints)}

# Forward shift: transcriptome at t correlates with microbiome at t+1 (transcriptome leads)
circ2_trans_shifted_forward = circ2_transcriptome_by_tp.copy()
circ2_trans_shifted_forward.index = [
    circ2_idx_to_tp.get(circ2_tp_to_idx[tp] + 1, None)
    for tp in circ2_trans_shifted_forward.index
]
circ2_trans_shifted_forward = circ2_trans_shifted_forward[
    circ2_trans_shifted_forward.index.notna()
]

# Backward shift: transcriptome at t correlates with microbiome at t-1 (microbiome leads)
circ2_trans_shifted_backward = circ2_transcriptome_by_tp.copy()
circ2_trans_shifted_backward.index = [
    circ2_idx_to_tp.get(circ2_tp_to_idx[tp] - 1, None)
    for tp in circ2_trans_shifted_backward.index
]
circ2_trans_shifted_backward = circ2_trans_shifted_backward[
    circ2_trans_shifted_backward.index.notna()
]

print(f"Forward shifted timepoints: {list(circ2_trans_shifted_forward.index)}")
print(f"Backward shifted timepoints: {list(circ2_trans_shifted_backward.index)}")

In [ ]:
# Pivot microbiome data for Circadian 2
circ2_microbiome_by_tp = circ2_timepoint_avg_abundance.pivot(
    index="timepoint", columns="Genus", values="AbundR100"
)

circ2_microbiome_by_tp = circ2_microbiome_by_tp.loc[
    :, (circ2_microbiome_by_tp > 0).sum() > minimum_samples_threshold_circadian
]
circ2_microbiome_by_tp = circ2_microbiome_by_tp.sort_index()

# Calculate correlations for Circadian 2
print("=== Circadian 2: No shift (synchronous) ===")
circ2_corr_no_shift, circ2_pval_no_shift, circ2_tp_no_shift = (
    calculate_gene_genus_correlations(circ2_transcriptome_by_tp, circ2_microbiome_by_tp)
)

print("\n=== Circadian 2: Forward shift (transcriptome leads) ===")
circ2_corr_forward, circ2_pval_forward, circ2_tp_forward = (
    calculate_gene_genus_correlations(
        circ2_trans_shifted_forward, circ2_microbiome_by_tp
    )
)

print("\n=== Circadian 2: Backward shift (microbiome leads) ===")
circ2_corr_backward, circ2_pval_backward, circ2_tp_backward = (
    calculate_gene_genus_correlations(
        circ2_trans_shifted_backward, circ2_microbiome_by_tp
    )
)

In [ ]:
# Circadian 2: Compare correlation distributions
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

circ2_corr_no_shift_flat = circ2_corr_no_shift.values.flatten()
circ2_corr_forward_flat = circ2_corr_forward.values.flatten()
circ2_corr_backward_flat = circ2_corr_backward.values.flatten()

circ2_corr_no_shift_flat = circ2_corr_no_shift_flat[~np.isnan(circ2_corr_no_shift_flat)]
circ2_corr_forward_flat = circ2_corr_forward_flat[~np.isnan(circ2_corr_forward_flat)]
circ2_corr_backward_flat = circ2_corr_backward_flat[~np.isnan(circ2_corr_backward_flat)]

axes[0].hist(circ2_corr_no_shift_flat, bins=50, alpha=0.7, edgecolor="black")
axes[0].set_title("Circadian 2: No Shift")
axes[0].set_xlabel("Spearman Correlation")
axes[0].axvline(x=0, color="red", linestyle="--", alpha=0.5)

axes[1].hist(
    circ2_corr_forward_flat, bins=50, alpha=0.7, edgecolor="black", color="green"
)
axes[1].set_title("Circadian 2: Forward Shift\n(Transcriptome leads)")
axes[1].set_xlabel("Spearman Correlation")
axes[1].axvline(x=0, color="red", linestyle="--", alpha=0.5)

axes[2].hist(
    circ2_corr_backward_flat, bins=50, alpha=0.7, edgecolor="black", color="orange"
)
axes[2].set_title("Circadian 2: Backward Shift\n(Microbiome leads)")
axes[2].set_xlabel("Spearman Correlation")
axes[2].axvline(x=0, color="red", linestyle="--", alpha=0.5)

plt.suptitle("Circadian Experiment 2 - Correlation Distributions", y=1.02)
plt.tight_layout()
plt.show()

print(
    f"Circadian 2 - No shift mean abs correlation: {np.mean(np.abs(circ2_corr_no_shift_flat)):.4f}"
)
print(
    f"Circadian 2 - Forward shift mean abs correlation: {np.mean(np.abs(circ2_corr_forward_flat)):.4f}"
)
print(
    f"Circadian 2 - Backward shift mean abs correlation: {np.mean(np.abs(circ2_corr_backward_flat)):.4f}"
)

In [ ]:
# # Save Circadian 2 results
# output_dir_circ2 = "/Users/michael/Data/Luke_terrace_experiment/General_data/Junk_data/Time_shifted_correlation/Circadian_2"
# os.makedirs(output_dir_circ2, exist_ok=True)

# dbfile = open(f"{output_dir_circ2}/corr_no_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_corr_no_shift, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ2}/pval_no_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_pval_no_shift, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ2}/tp_no_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_tp_no_shift, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ2}/corr_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_corr_forward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ2}/pval_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_pval_forward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ2}/tp_forward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_tp_forward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ2}/corr_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_corr_backward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ2}/pval_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_pval_backward, dbfile)
# dbfile.close()

# dbfile = open(f"{output_dir_circ2}/tp_backward_shift_remove_low_exp.p", "ab")
# pickle.dump(circ2_tp_backward, dbfile)
# dbfile.close()

print(f"Saved Circadian 2 results to {output_dir_circ2}")